In [ ]:
import pandas as pd 
import torch
from plotnine import *

from vpop_calibration import *

%load_ext autoreload
%autoreload 2


In [ ]:
# Setup the training data for the surrogate model
# An analytical expression is available for the model


def logistic_growth(lambda1, lambda2, lambda3, t):
    """Analytical expression of a logistic growth model

    Args:
        t: age in days
        lambda1, lambda2, lambda3: growth parameters

    Returns:
        y: Predicted concentration
    """
    y = lambda1 / (1 + torch.exp(-(t - lambda2) / lambda3))
    return y


struct_model = StructuralAnalytical(
    logistic_growth, ["lambda1", "lambda2", "lambda3"], ["circumference"]
)

In [ ]:
orange_trees_data_url = "https://raw.githubusercontent.com/vincentarelbundock/Rdatasets/refs/heads/master/csv/datasets/Orange.csv"

df = pd.read_csv(orange_trees_data_url)

display(df.head())

trees_df = df[["Tree"]].drop_duplicates().rename(columns={"Tree": "id"})
display(trees_df.head())

obs_df = (
    df.copy()
    .drop(columns=["rownames"])
    .rename(columns={"Tree": "id", "age": "time", "circumference": "value"})[
        ["id", "time", "value"]
    ]
    .astype({"value": "float", "time": "float"})
)
obs_df["output_name"] = "circumference"
obs_df["protocol_arm"] = "identity"
display(obs_df.head())

In [ ]:
init_log_mi = {"lambda2": 0.0, "lambda3": 0.0}
init_log_pdu = {
    "lambda1": {"mean": 0.0, "sd": 0.5},
}
covariate_map = None
init_res_var = [100.0]
nlme_model = NlmeModel(
    structural_model=struct_model,
    patients_df=trees_df,
    init_log_MI=init_log_mi,
    init_PDU=init_log_pdu,
    covariate_map=covariate_map,
    init_res_var=init_res_var,
    error_model_type="additive",
    num_chains=1,
    constraints={
        "lambda1": {"low": 100, "high": 600},
        "lambda3": {"low": 100, "high": 600},
    },
)

optimizer = PySaem(
    model=nlme_model,
    observations_df=obs_df,
    mcmc_first_burn_in=100,
    mcmc_nb_transitions=3,
    nb_phase1_iterations=500,
    nb_phase2_iterations=500,
    plot_frames=200,
    verbose=False,
)

In [ ]:
optimizer.run()

In [ ]:
plot_map_estimates(nlme_model)

In [ ]:
plot_weighted_residuals(nlme_model, "iwres")

In [ ]:
plot_weighted_residuals(nlme_model, "pwres")

In [ ]:
plot_weighted_residuals(nlme_model, "npde")

In [ ]:
plot_map_vs_posterior(nlme_model, 5000, 3)